In [1]:
import math
from datetime import datetime
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, accuracy_score, recall_score, precision_score, f1_score
import random as python_random

#pip install image-classifiers==1.0.0b1
from classification_models.tfkeras import Classifiers

In [2]:

np.random.seed(2021)
python_random.seed(2021)
tf.random.set_seed(2021)

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(mirrored_strategy.num_replicas_in_sync))

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [4]:
data_df = pd.read_csv('2021_04_04_asian-black-white-chexpert_ver_A.csv')
data_df = data_df[data_df.ETHNICITY=="Non-Hispanic/Non-Latino"]
data_df = data_df[data_df.Position.isin(['AP','PA'])]

In [5]:
data_df.split.value_counts(normalize=True)

train       0.601337
test        0.302002
validate    0.096660
Name: split, dtype: float64

In [6]:
data_df.race.value_counts(normalize=True)

WHITE                     0.779041
ASIAN                     0.148145
BLACK/AFRICAN AMERICAN    0.072814
Name: race, dtype: float64

In [7]:
train_df = data_df[data_df.split=="train"]
validation_df = data_df[data_df.split=="validate"]
test_df = data_df[data_df.split=="test"]

In [8]:
#False indicates no patient_id shared between groups

unique_train_id = train_df.patient_id.unique()
unique_validation_id = validation_df.patient_id.unique()
unique_test_id = test_df.patient_id.unique()
all_id = np.concatenate((unique_train_id, unique_validation_id, unique_test_id), axis=None)

def contains_duplicates(X):
    return len(np.unique(X)) != len(X)

contains_duplicates(all_id)

False

In [9]:
HEIGHT, WIDTH = 320, 320

arc_name = "CHEXPERT-" + str(HEIGHT) + "x" + str(WIDTH) + "_60-10-30-split-resnet34-Float16_3-race_detection"

In [10]:
resnet34, preprocess_input = Classifiers.get('resnet34')

In [11]:
with mirrored_strategy.scope():
    input_a = Input(shape=(HEIGHT, WIDTH, 3))
    base_model = resnet34(input_tensor=input_a, include_top=False, input_shape=(HEIGHT,WIDTH,3), weights='imagenet')
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(3, name='dense_logits')(x)
    output = Activation('softmax', dtype='float32', name='predictions')(x)
    model = Model(inputs=[input_a], outputs=[output])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [12]:
learning_rate = 1e-3
momentum_val=0.9
decay_val= 0.0
train_batch_size = 256 # may need to reduce batch size if OOM error occurs
test_batch_size = 256

reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=2, min_lr=1e-5, verbose=1)

adam_opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=decay_val)
adam_opt = tf.keras.mixed_precision.LossScaleOptimizer(adam_opt)

with mirrored_strategy.scope():
    model.compile(optimizer=adam_opt,
                    loss=tf.losses.CategoricalCrossentropy(),
                    metrics=[
                        tf.keras.metrics.AUC(curve='ROC', name='ROC-AUC'),
                        tf.keras.metrics.AUC(curve='PR', name='PR-AUC')
                    ],
    )

In [13]:
train_gen = ImageDataGenerator(
            rotation_range=15,
            fill_mode='constant',
            horizontal_flip=True,
            zoom_range=0.1,
            preprocessing_function=preprocess_input
            )

validate_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [14]:
train_batches = train_gen.flow_from_dataframe(train_df, directory="/tf/notebooks/SSD_data/chexpert_directory/", x_col="Path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=True,seed=2021,batch_size=train_batch_size, dtype='float32')
validate_batches = validate_gen.flow_from_dataframe(validation_df, directory="/tf/notebooks/SSD_data/chexpert_directory/", x_col="Path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=test_batch_size, dtype='float32')        

Found 76433 validated image filenames belonging to 3 classes.
Found 12286 validated image filenames belonging to 3 classes.


In [15]:
train_epoch = math.ceil(len(train_df) / train_batch_size)
val_epoch = math.ceil(len(validation_df) / test_batch_size)

In [16]:
var_date = datetime.now().strftime("%Y%m%d-%H%M%S")
ES = EarlyStopping(monitor='val_loss', mode='min', patience=4, restore_best_weights=True)
checkloss = ModelCheckpoint("../saved_models/racial_bias/trials/" + str(arc_name) + "_CXR_LR-" + str(learning_rate) + "_" + var_date+"_epoch:{epoch:03d}_val_loss:{val_loss:.5f}.hdf5", monitor='val_loss', mode='min', verbose=1, save_best_only=True, save_weights_only=False)

In [17]:
model.fit(train_batches,
            validation_data=validate_batches,
            epochs=100,
            steps_per_epoch=int(train_epoch),
            validation_steps=int(val_epoch),
            workers=32,
            max_queue_size=50,
            shuffle=False,
            callbacks=[checkloss, reduce_lr, ES]
           )

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 108 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 108 all-reduces with algorithm = nccl, num_packs = 1
299/299 [==============================] - 781s 2s/step - loss: 0.4739 - ROC-AUC: 0.9392 - PR-AUC: 0.8933 - val_loss: 3.5495 - val_ROC-AUC: 0.4986 - val_PR-AUC: 0.3073

Epoch 00001: val_loss improved from inf to 3.54947, saving model to ../saved_models/racial_bias/trials/CHEXPERT-320x320_60-10-30-split-resnet34-Float16_3-race_detection_CXR_LR-0.001_20210627-134953_epoch:001_val_loss:3.54947.hdf5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
299/299 [==============================] - 594s 2s/step - loss: 0.3289 - ROC-AUC: 0.9697 - PR-AUC: 0.9454 - val_loss: 0.3562 - val_ROC-AUC: 0.9683 - val_PR-AUC: 0.9446

Epoch 00002: val_loss improved from 3.54947 to 0.35616, saving model to ../saved_models/racial_bias/trials/CHEXPERT-320x320_60-10-30-split-resnet34-Float16_3-race_detection_CXR_LR-0.001_20210627-134953_epoch:002_val_loss:0.35616.hdf5
Epoch 3/100
299/299 [==============================] - 563s 2s/step - loss: 0.2811 - ROC-AUC: 0.9774 - PR-AUC: 0.9588 - val_loss: 0.3598 - val_ROC-AUC: 0.9701 - val_PR-AUC: 0.9477

Epoch 00003: val_loss did not improve from 0.35616
Epoch 4/100
299/299 [==============================] - 555s 2s/step - loss: 0.2530 - ROC-AUC: 0.9813 - PR-AUC: 0.9660 - val_loss: 0.3989 - val_ROC-AUC: 0.9611 - val_PR-AUC: 0.9338

Epoch 00004: val_loss did not improve from 0.35616

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/100
299/299 [==================

In [18]:
test_batches = validate_gen.flow_from_dataframe(test_df, directory="/tf/notebooks/fishtank/radiology_datasets/CheXpert_Xray_dataset/resize_chexpert_320x320/chexpert_data/", x_col="Path", y_col="race", class_mode="categorical",target_size=(HEIGHT, WIDTH),shuffle=False,batch_size=test_batch_size, dtype='float32')        

Found 38386 validated image filenames belonging to 3 classes.


In [19]:
with mirrored_strategy.scope():

    multilabel_predict_test = model.predict(test_batches, max_queue_size=10, verbose=1, steps=math.ceil(len(test_df)/test_batch_size), workers=16)


150/150 [==============================] - 134s 873ms/step


In [20]:
input_prediction = multilabel_predict_test
input_df = test_df
input_prediction_df = pd.DataFrame(input_prediction)
true_logits = pd.DataFrame()
loss_log = pd.DataFrame()

In [21]:
def stat_calc(input_prediction_df, input_df):
    ground_truth = input_df.race
    pathology_array=[
        'ASIAN',
        'BLACK/AFRICAN AMERICAN',
        'WHITE'
        ]
    i=0
    auc_array = []
    for pathology in pathology_array:
    
        new_truth = (ground_truth.str.contains(pathology)).apply(int)
        input_prediction_val = input_prediction_df[i]
        val = input_prediction_val
        AUC = roc_auc_score(new_truth, val)
        true_logits.insert(i, i, new_truth, True)
        auc_array.append(AUC)
        i += 1
        
    progress_df = pd.DataFrame({'Study':pathology_array, 'AUC':auc_array})
    print(progress_df)

In [22]:
stat_calc(input_prediction_df, input_df)

                    Study       AUC
0                   ASIAN  0.970054
1  BLACK/AFRICAN AMERICAN  0.967394
2                   WHITE  0.966309
